In [20]:
# Extract first 500 queries from training data
import csv

with open('collections/queries.train.tsv','r') as tsvIn, open('collections/queries.train.500.tsv', 'w') as tsvOut:
    reader = csv.reader(tsvIn, delimiter='\t')
    writer = csv.writer(tsvOut, delimiter='\t')
    i=1;
    for row in reader:
        if (i<=500):
            writer.writerow(row)     
        i+=1

In [21]:
# Load 500 queries into topics500train
import csv

with open('collections/queries.train.500.tsv','r') as tsvIn:
    reader = csv.reader(tsvIn, delimiter='\t')
    topics500train = {int(row[0]): {"title": row[1]} for row in reader}
    print(len(topics500train))

500


In [18]:
# Load msmarco-passage index into Pyserini searcher object
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

Attempting to initialize pre-built index msmarco-passage.
/Users/louiseleibbrandt/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d already exists, skipping download.
Initializing msmarco-passage...


In [36]:
# Run search at rank 1000
def run_all_queries(file, topics, searcher):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in topics:
            query = topics[id]['title']
            hits = searcher.search(query, 1000)
            for i in range(0, len(hits)):
                _ = runfile.write('{}\t{}\t{}\n'.format(id, hits[i].docid, i+1))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')

# Set BM25 parameters to default values
searcher.set_bm25(0.9, 0.4)
run_all_queries('runs/run-msmarco-500testqueries-bm25.txt', topics500train, searcher)

Running 500 queries in total
100 queries completed
200 queries completed
300 queries completed
400 queries completed
500 queries completed


In [37]:
# write run to csv
import pandas as pd

read_file = pd.read_csv (r'runs/run-msmarco-500testqueries-bm25.txt')
read_file.to_csv(r'runs/run-msmarco-500testqueries-bm25.tsv', index=None)


In [41]:
# MRR at rank 10 for all 500 queries (using original msmarco_passage_eval file)
!python eval/msmarco_passage_eval.py \
 collections/qrels.train.tsv runs/run-msmarco-500testqueries-bm25.tsv

Queries Ranked: 500
#####################
MRR @10: 0.11122460317460323
QueriesRanked: 500
#####################


In [43]:
# MRR at rank 100 for each 500 queries (using our modified msmarco_passage_eval_modified file)
!python eval/msmarco_passage_eval_modified.py \
 collections/qrels.train.tsv runs/run-msmarco-500testqueries-bm25.tsv

121352 0
634306 0
920825 0.0136986301369863
510633 0
737889 1.0
674172 0
303205 0
570009 0
492875 1.0
54528 0.06666666666666667
738368 0.3333333333333333
507001 1.0
466926 0.07142857142857142
1181095 0.06666666666666667
224811 0.0625
918533 0
80926 0
906071 0
428191 0.5
790457 0.5
242081 0
1006418 1.0
843270 0
551231 0
166621 0.5
490462 0.04
316859 0
45248 0.022222222222222223
1176220 0.023255813953488372
867974 0.015384615384615385
529604 0
439538 0.5
436798 0.3333333333333333
1074527 0.5
167374 0.3333333333333333
389298 0
640699 0
132366 1.0
932727 0
466858 0.3333333333333333
1079617 1.0
381728 0
672091 0.1111111111111111
895696 0.5
4801 0.5
122479 0.038461538461538464
1180498 0
797201 0.018867924528301886
293603 1.0
547843 0
1057322 0
1168693 0.5
641469 0
693178 0.05
23229 0.04
874646 0.05
924673 0.2
207697 0
445243 0.5
291569 0.015625
493513 1.0
506152 0.25
322801 0.14285714285714285
953121 0.14285714285714285
881296 0.3333333333333333
1142466 1.0
20627 0.125
1146849 1.0
2341 0
495

# Q1 average fee to put an animal to sleep?
## Passages Retrieved by BM25 default param
### Passage 1 Retrieved: 6342346
Euthanasia literally means 'gentle death'. Other terms you may hear are 'put to sleep', 'put down', 'put out of its misery' or, less kindly, 'destroy'. Veterinary staff may use the term 'humane destruction' which is simply a technical term for putting an animal to sleep. The decision to end a life is never easy.

### Passage 2 Retrieved: 4349746
Damage to the pons will result in a range of effects on sleep. For example, damage to certain parts of the pons can put a person into a semi-permanent sleep-like state. Resear â€¦ ch with animals has also shown that damage to the pons can cause an animal to sleep much less than prior to the damage.

### Passage 3 Retrieved: 1562117
Definition of put to sleep. 1 : to give (a sick or injured animal) drugs that will make it die without pain. 2 informal : to use a drug to make (someone) unconscious before a medical operation : to give (someone) anesthesia before a medical operation. 3 : to get (someone) ready to sleep for the night Did you put the kids to sleep?

## Relevant Passage (as defined by qrel file): 885367
Rating Newest Oldest. Best Answer: The cost depends upon many factors. When my dog got old and ill, the vet put him to sleep and didn't even charge, because we went to that vet for many years. When my first cat reached 21 and developed hypertension, we had her put to sleep. I think it cost about 60.00.